<a href="https://colab.research.google.com/github/Ayushsrinivas7/LLM_LEARNINGS/blob/main/final_BUILDING_THE_AGENT_FROM_SCRATCH_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install groq

In [44]:
import os
os.environ['GROQ_API_KEY'] = "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"


In [45]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature = 0
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, which is essential for applications that require real-time or near-real-time processing, such as chatbots, virtual assistants, and sentiment analysis tools.
2. **Improved User Experience**: Fast language models enable applications to respond quickly to user queries, improving the overall user experience. This is particularly important for applications that require immediate feedback, such as language translation, text summarization, and question-answering systems.
3. **Scalability**: Fast language models can handle a large volume of requests and data, making them ideal for large-scale applications, such as social media platforms, search engines, and content management systems.
4. **Reduced Latency**: Fast language models minimize latency, which is

In [46]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content



In [47]:
system_prompt1 = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


In [48]:
system_prompt2 = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: (5 + 3) * 2
Runs a calculation and returns the number - uses Python, so be sure to use floating point syntax if necessary.

wikipedia:
e.g. wikipedia: Albert Einstein
Search Wikipedia and return a summary for the given term.

book_summary:
e.g. book_summary: Harry Potter and the Sorcerer’s Stone
Searches a book database and returns a short summary of the specified book.

Example sessions:

---

**Session 1: Calculate**

Question: How many pages is 10 chapters if each chapter is 25 pages?
Thought: I should calculate the total pages by multiplying chapters by pages per chapter.
Action: calculate: 10 * 25
PAUSE

You will be called again with this:

Observation: 250

You then output:

Answer: The total number of pages is 250.

---

**Session 2: Wikipedia**

Question: Who is Albert Einstein?
Thought: I should look up Albert Einstein on Wikipedia to find more details about him.
Action: wikipedia: Albert Einstein
PAUSE

You will be called again with this:

Observation: Albert Einstein was a theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics. He is best known for his mass-energy equivalence formula, E = mc².

You then output:

Answer: Albert Einstein was a theoretical physicist who developed the theory of relativity and is best known for his formula, E = mc².

---

**Session 3: Book Summary**

Question: Can you summarize "Harry Potter and the Sorcerer’s Stone"?
Thought: I should use the book_summary action to get a summary of the book.
Action: book_summary: Harry Potter and the Sorcerer’s Stone
PAUSE

You will be called again with this:

Observation: "Harry Potter and the Sorcerer's Stone" is the first book in the Harry Potter series by J.K. Rowling. It introduces Harry, an orphan who discovers he is a wizard on his 11th birthday, and follows his adventures at Hogwarts School of Witchcraft and Wizardry.

You then output:

Answer: "Harry Potter and the Sorcerer's Stone" is the first book in the series, introducing Harry as he discovers he is a wizard and embarks on adventures at Hogwarts School of Witchcraft and Wizardry.

---------

Now it's your turn :
""".strip()

In [49]:
system_prompt3 = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris

Now it's your turn :

""".strip()


In [50]:
system_prompt4 = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this:

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [51]:



import re
import httpx


def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def calculate(operation: str) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float :
   planet = planet.lower()
   if planet ==  "earth":
            return 5.972e24
   elif planet ==  "jupiter":
            return 1.898e27
   elif planet ==  "mars":
            return 6.39e23
   elif planet ==  "mercury":
            return 3.285e23
   elif planet ==  "neptune":
            return 1.024e26
   elif planet ==  "saturn":
            return 5.683e26
   elif planet ==  "uranus":
            return 8.681e25
   elif planet ==  "venus":
            return 4.867e24
   else:

            return 0.0

In [52]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt4)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


In [53]:
loop(query="What is current age of Mr. Nadrendra Modi multiplied by 2?")


Thought: I need to find the current age of Narendra Modi on Wikipedia. I'll also need to find his birthdate to calculate his current age.
Action: wikipedia: Narendra Modi
PAUSE
[('wikipedia', 'Narendra Modi')]
Observation: <span class="searchmatch">Narendra</span> Damodardas <span class="searchmatch">Modi</span> (born 17 September 1950) is an Indian politician who has served as Prime Minister of India since 2014. <span class="searchmatch">Modi</span> was the chief minister
Thought: I've found Narendra Modi's birthdate, which is 17 September 1950. Now I need to calculate his current age.
Action: calculate: 2023 - 1950
PAUSE
[('calculate', '2023 - 1950')]
Observation: 73
Thought: I've found Narendra Modi's current age, which is 73. Now I need to multiply it by 2.
Action: calculate: 73 * 2
PAUSE
[('calculate', '73 * 2')]
Observation: 146
Thought: I've calculated Narendra Modi's current age multiplied by 2, which is 146.
Answer: The current age of Mr. Narendra Modi multiplied by 2 is 146.


In [54]:
loop(query="What will be age of Mr. Narendra Modi in 2024 multiplied by 2?")

Thought: I need to find the birthdate of Narendra Modi on Wikipedia
Action: wikipedia: Narendra Modi
PAUSE
[('wikipedia', 'Narendra Modi')]
Observation: <span class="searchmatch">Narendra</span> Damodardas <span class="searchmatch">Modi</span> (born 17 September 1950) is an Indian politician who has served as Prime Minister of India since 2014. <span class="searchmatch">Modi</span> was the chief minister
Thought: I have found the birthdate of Narendra Modi, now I need to calculate his age in 2024 and multiply it by 2.
Action: calculate: (2024 - 1950) * 2
PAUSE
[('calculate', '(2024 - 1950) * 2')]
Observation: 148
Thought: I think I have found the answer
Action: The age of Narendra Modi in 2024 multiplied by 2 is 148.
Answer: The age of Narendra Modi in 2024 multiplied by 2 is 148.


checking the flow